In [ ]:
# We start by importing necessary libraries.
import pathlib
import sys
import glob
import pandas as pd
import geopandas as gpd
# Here's the popexposure import 
import popexposure  

In [ ]:
base_path = pathlib.Path.cwd().parent
data_dir = base_path / "demo_data"

In [ ]:
# read in US wildfire dataset
fires = gpd.read_file(data_dir / "01_raw_data"/ "wfbz_disasters_conus.geojson")
# filter to only CA fires - wildfire_states has to contain CA
fires = fires[fires['wildfire_states'].str.contains('CA')]

# read in US wildfire dataset
fires = gpd.read_file(data_dir / "01_raw_data"/ "wfbz_disasters_conus.geojson")
# filter to only CA fires - wildfire_states has to contain CA
fires = fires[fires['wildfire_states'].str.contains('CA')]

In [ ]:
border_fires = fires[fires['wildfire_id'].str.contains('camp', case=False, na=False)].copy()

In [ ]:
border_fires

In [ ]:
border_fires["buffer_dist_10"] = 0  # buffer distance in meters 
border_fires.head()

In [ ]:
# First, select cols
by_year = border_fires[["wildfire_id", "buffer_dist_10", "geometry"]]
# Then rename the wildfire ID col
by_year = by_year.rename(columns={"wildfire_id": "ID_hazard"})

In [ ]:
by_year

In [ ]:
# save by year in the data
border_fires = data_dir / "02_interim_data" / "border_fire.geojson"
by_year.to_file(border_fires, driver="GeoJSON")

In [ ]:
# run popexp on it
ghsl_path = data_dir / "01_raw_data" / "GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0_R5_C8.tif"
# ZCTA path 
zcta_path = glob.glob(str(data_dir / "02_interim_data" / "*zcta*"))[0]

In [ ]:
est = popexposure.PopEstimator()

year_dat = est.prep_data(path_to_data=border_fires, geo_type='hazard')
num_exposed = est.est_exposed_pop(pop_path=ghsl_path, 
                                  hazards = year_dat,
                                  hazard_specific=True)

In [ ]:
num_exposed